# Issues, and lots of them


Filter type not properly working - returning kn95 false positives
earloop not properly working - returning adjustable false positives


user-agent times out after a few hundred iterations of retrieving information, probably an anti-bot measure from amazon


when running the looped search results section, all size, color, gender, material, filter type, earloop info is identical and is not calculating correctly

# This is a test code for an individual item's webpage
This must be run first!!


In [ ]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
# File = open("maskData.csv", "a")

#get title
def title(soup):
  try:
    itemTitle = soup.find("span", attrs = {"id": 'productTitle'}).string.strip()
  except AttributeError:
    itemTitle = ""
  return itemTitle

#get price
def price(soup):
  try:
    itemPrice = soup.find("div", attrs = {'id': 'priceblock_ourprice'}).string.strip()
  except AttributeError:
    itemPrice = ""
  if not itemPrice:
    try:
      itemPrice = soup.find("span", attrs = {'id': 'priceblock_dealprice'}).string.strip()
    except AttributeError:
      itemPrice = ""
  return itemPrice

#get ratings
def rating(soup):
  try:
    itemRating = soup.find("i", attrs = {'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
  except AttributeError:
    try:
      itemRating = soup.find("span", attrs = {'class': 'a-icon-alt'}).string.strip()
    except:
      itemRating = ""
  return itemRating

#get rating count
def ratingCount(soup):
  try:
    rateCount = soup.find("span", attrs = {'id': 'acrCustomerReviewText'}).string.strip()
  except AttributeError:
    rateCount = ""
  return rateCount

#get stock
def inStock(soup):
  try:
    stock = soup.find("div", attrs = {'id': 'availability'}).find("span").string.strip()
  except AttributeError:
    stock = ""
  return stock

#get description
def description(soup):
  try:
    descript = []
    descriptionStr = ""
    if len(soup.select("#feature-bullets ul.a-unordered-list")) > 0:
      for line in soup.select("#feature-bullets ul.a-unordered-list")[0].find_all('li'):
          descript.append(line.get_text().strip())
      if descript[0] == 'Make sure this fits\nby entering your model number.':
        descript.pop(0)
      for item in descript:
        descriptionStr = descriptionStr + ' ' +  item
    else:
      descriptionStr = ""
  except AttributeError:
    descriptionStr = ""
  return descriptionStr

#get size
def size(title, description):
  fullStr = title.lower() + description.lower()
  sizeFilters = ['universal', 'large', 'small', 'medium', 'x-large', 'youth', 'one size fits most', 'adult', 'kid', 'suitable for most', 'children', 'boy', 'girl']
  mark = False

  for item in sizeFilters:
    if item in fullStr:
      mark = True
  if mark == False:
    size = 'universal'
    return size
  if 'children' in fullStr or 'kid' in fullStr or 'youth' in fullStr or 'boys' in fullStr or 'girls' in fullStr:
    size = 'childrens'
    return size
  size = ''
  for item in sizeFilters:
    if item in fullStr:
      size += item + ", "
  if not size:
    return 'NA'
  return size

#get color
def color(title, description):
  fullStr = title.lower() + description.lower()
  colorFilters = ['black', 'blue', 'colored', 'colorful', 'navy', 'red', 'plaid', 'peach', 'tie-dye', 'tie die', 'pink', 'purple', 'green', 'yellow', 'floral', 'gray', 'grey', 'burgundy', 'khaki', 'army']
  n = 0
  for item in colorFilters:
    if item in fullStr:
      n += 1
    if n > 1:
      return 'various'
  for item in colorFilters:
    if item in fullStr:
      return item
  if n == 0:
    return 'NA'

#get material
def material(title, description):
  fullStr = title.lower() + description.lower()
  materialFilters = ['non-woven', 'non woven', 'polypropylene', 'melt-spray', 'melt spray', 'polyester', 'cotton', 'melt-blown', 'melt blown', 'hypoallergenic', 'polyester zinc', 'spandex']
  n = 0
  if materialFilters[0] in fullStr or materialFilters[1] in fullStr or materialFilters[3] in fullStr or materialFilters[4] in fullStr or materialFilters[8] in fullStr or materialFilters[9] in fullStr:
    return 'non-woven melt-spray'
  for item in materialFilters:
    if item in fullStr:
      n += 1
    if n > 1:
      return 'mixed material'
  for item in materialFilters:
    if item in fullStr:
      return item
  if n == 0:
    return 'NA'

#get filter type
def filterType(title, description):
  fullStr = title.lower() + description.lower()
  typeFilters = ['3-ply', '3 ply', '3-layers', 'filter pocket', 'kn95', '5-ply', '5 ply', '4-ply', '4 ply', 'pm2.5', 'pm 2.5', '3 layer', '2-ply', '2 ply', 'n95']
  if 'kn95' in fullStr or 'kn 95' in fullStr:
    return 'kn95'
  if 'n95' in fullStr:
    return 'n95'
  if 'filter pocket' in fullStr or 'pm2.5' in fullStr or 'pm 2.5' in fullStr:
    return 'filter pocket'
  for item in typeFilters:
    if item in fullStr:
      return item
  return 'NA'

  # designFilters = ['solid', 'pattern', 'printed']

#get gender
def gender(title, description):
  fullStr = title.lower() + description.lower()
  genderFilter = ['women', 'woman', 'for women', 'unisex', 'men', 'man', 'boy', 'girl']
  if (' men' in fullStr and 'women' in fullStr) or ('woman' in fullStr and ' man' in fullStr) or ('boy' in fullStr and 'girl' in fullStr) or 'unisex' in fullStr:
    return 'unisex'
  if 'female' in fullStr or 'woman' in fullStr or 'women' in fullStr:
    return 'womens'
  if 'male' in fullStr or ' men' in fullStr or ' man' in fullStr:
    return 'mens'
  if 'boy' in fullStr:
    return 'boys'
  if 'girl' in fullStr:
    return 'girls'
  return 'unisex'

#get disposability
def disposability(title, description):
  fullStr = title.lower() + description.lower()
  disposabilityFilter = ['disposable', 'machine washable', 'reusable', 'single use', 'hand wash', 'machine wash']
  if 'disposable' in fullStr or 'single use' in fullStr or 'single-use' in fullStr:
    return 'disposable'
  if 'machine washable' in fullStr or 'reusable' in fullStr or 'machine-washable' in fullStr or 'hand wash' in fullStr:
    return 'reusable'
  return 'NA'

#get earloop
def earloop(title, description):
  fullStr = title.lower() + description.lower()
  earFilter = ['elastic', 'stretch', 'adjustable ear', 'stopper']
  if 'adjustable ear' in fullStr or 'stopper' in fullStr:
    return 'adjustable'
  if 'elastic' in fullStr or 'stretch' in fullStr:
    return 'elastic'
  return 'NA'


if __name__ == '__main__':
 
    # Headers for request, basically an antibot measure
    head = ({'User-Agent':
                'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; KTXN)',
                'Accept-Language': 'en-US, en;q=0.5'})
 
    # webpage URL
    URL = "https://www.amazon.com/Craft-Soul-Reusable-Washable-Comfortable/dp/B08BDG1H4R/ref=sr_1_55?dchild=1&keywords=face+mask&qid=1618306480&s=hpc&sr=1-55"
 
    # HTTP Request
    webpage = rq.get(URL, headers=head)
 
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "lxml")

    # displaying all info
    print("Product Title =", title(soup))
    # print("Product Category =", category(soup))
    print("Product Price =", price(soup))
    print("Product Rating =", rating(soup))
    print("Number of Product Reviews =", ratingCount(soup))
    print("Availability =", inStock(soup))
    print("Description = ", description(soup))
    print("Size = ", size(title(soup), description(soup)))
    print("Color = ", color(title(soup), description(soup)))
    print("Material = ", material(title(soup), description(soup)))
    print("Filter type = ", filterType(title(soup), description(soup)))
    print("Gender = ", gender(title(soup), description(soup)))
    print("Disposability = ", disposability(title(soup), description(soup)))
    print("Ear loop style = ", earloop(title(soup), description(soup)))
    print()
    print()


    # #BeautifulSoup find all links similar
    # linkSoup = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
    # #create a list of links from links
    # links = []
    # for link in linkSoup:
    #   links.append(link.get('href'))

    # #extract title, price, rating, # of ratings, and availability from links
    # for link in links:
    #   item = rq.get("https://www.amazon.com" + link, headers = head)
    #   newSoup = BeautifulSoup(item.content, "lxml")
           
    #   print("Product Title =", title(new_soup))
    #   print("Product Price =", price(new_soup))
    #   print("Product Rating =", rating(new_soup))
    #   print("Number of Product Reviews =", ratingCount(new_soup))
    #   print("Availability =", inStock(new_soup))


Product Title = (5 Pack Pieces per Box) Craft & Soul Mattana Face Mask Cover Reusable Washable Comfortable Fabric Cloth
Product Price = $11.90
Product Rating = 4.3 out of 5 stars
Number of Product Reviews = 9,911 ratings
Availability = In Stock.
Description =   General Use - This is a general purpose face mask. Not intended for medical use. Each box contains 5 pieces. Made In Vietnam. Quality Construction - The quality fabric, stitching, ear straps, and fit are designed for comfort and durability. Made by the same skilled apparel workers that make Craft & Soul Men's Clothing. Craft & Soul is a U.S. based brand with manufacturing in Vietnam. Craft & Soul is available exclusively on Amazon. Comfortable - Made of high quality 2-Ply 35% Cotton/65% Poly blend knitted fabric. Soft texture and comfortable stretchable fabric that fits closely along the face. Breathable two layer design won't fog glasses. Washable & Reusable - Easy to care for and can be used repeatedly. One size which fits mos

# This scrapes the search results for 'face mask'

In [ ]:
if __name__ == '__main__':
 
    # Headers for request
    head = ({'User-Agent':
                'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; KTXN)',
                'Accept-Language': 'en-US'})
 
    # The search webpage URL for 'face masks'
    URL = "https://www.amazon.com/s?k=face+mask&i=hpc&ref=nb_sb_noss"
    URLs = ["https://www.amazon.com/s?k=face+mask&i=hpc&ref=nb_sb_noss", "https://www.amazon.com/s?k=face+mask&i=hpc&page=2&qid=1618280247&ref=sr_pg_2", "https://www.amazon.com/s?k=face+mask&i=hpc&page=3&qid=1618280251&ref=sr_pg_3", "https://www.amazon.com/s?k=face+mask&i=hpc&page=4&qid=1618280256&ref=sr_pg_4", "https://www.amazon.com/s?k=face+mask&i=hpc&page=5&qid=1618280293&ref=sr_pg_5", "https://www.amazon.com/s?k=face+mask&i=hpc&page=6&qid=1618280315&ref=sr_pg_6", "https://www.amazon.com/s?k=face+mask&i=hpc&page=7&qid=1618280350&ref=sr_pg_7", "https://www.amazon.com/s?k=face+mask&i=hpc&page=8&qid=1618280363&ref=sr_pg_8", "https://www.amazon.com/s?k=face+mask&i=hpc&page=9&qid=1618280368&ref=sr_pg_9", "https://www.amazon.com/s?k=face+mask&i=hpc&page=10&qid=1618280412&ref=sr_pg_10", "https://www.amazon.com/s?k=face+mask&i=hpc&page=11&qid=1618280426&ref=sr_pg_11", "https://www.amazon.com/s?k=face+mask&i=hpc&page=12&qid=1618280442&ref=sr_pg_12", "https://www.amazon.com/s?k=face+mask&i=hpc&page=13&qid=1618280451&ref=sr_pg_13", "https://www.amazon.com/s?k=face+mask&i=hpc&page=14&qid=1618280481&ref=sr_pg_14", "https://www.amazon.com/s?k=face+mask&i=hpc&page=15&qid=1618280499&ref=sr_pg_15", "https://www.amazon.com/s?k=face+mask&i=hpc&page=16&qid=1618280516&ref=sr_pg_16", "https://www.amazon.com/s?k=face+mask&i=hpc&page=17&qid=1618280528&ref=sr_pg_17", "https://www.amazon.com/s?k=face+mask&i=hpc&page=18&qid=1618280547&ref=sr_pg_18", "https://www.amazon.com/s?k=face+mask&i=hpc&page=19&qid=1618280559&ref=sr_pg_19", "https://www.amazon.com/s?k=face+mask&i=hpc&page=20&qid=1618280571&ref=sr_pg_20"]

    # # HTTP request
    # page = rq.get(URL, headers = head)

    # #soupy time!!!
    # soup = BeautifulSoup(page.content, "lxml")

    # #get list of links
    # linkSoup = soup.find_all("a", attrs = {'class' : 'a-link-normal s-no-outline'})
    # links = []
    # for link in linkSoup:
    #   links.append(link.get('href'))
    
    data = {'link': [], 'title': [], 'price': [], 'rating': [], 'ratingCount': [], 'stock': [], 'description': [], 'size': [], 'color': [], 'material': [], 'filter type': [], 'gender': [], 'disposability': [], 'earloop':[]}

    for url in URLs:
      # HTTP request
      page = rq.get(url, headers = head)

      #soupy time!!!
      soup = BeautifulSoup(page.content, "lxml")

      #get list of links
      linkSoup = soup.find_all("a", attrs = {'class' : 'a-link-normal s-no-outline'})
      links = []
      for link in linkSoup:
        links.append(link.get('href'))

      for link in links:
        if link == links[0]:
          continue
        item = rq.get("https://www.amazon.com" + link, headers = head)
        newSoup = BeautifulSoup(item.content, "lxml")

        data['link'].append(link)

        #title
        itemTitle = title(newSoup)
        print("Product Title =", itemTitle)
        # File.write(f"{itemTitle},")
        data['title'].append(itemTitle)

        #price
        itemPrice = price(newSoup)
        print("Product Price =", itemPrice)
        # File.write(f"{itemPrice},")
        data['price'].append(itemPrice)

        #rating
        itemRating = rating(newSoup)
        print("Product Rating =", itemRating)
        # File.write(f"{itemRating},")
        data['rating'].append(itemRating)

        #number of ratings
        itemRateCount = ratingCount(newSoup)
        print("Number of Product Reviews =", itemRateCount)
        # File.write(f"{itemRateCount},")
        data['ratingCount'].append(itemRateCount)

        #stock availability
        itemStock = inStock(newSoup)
        print("Availability =", itemStock)
        # File.write(f"{itemStock},")
        data['stock'].append(itemStock)      

        #description box of features
        itemDescription = description(newSoup)
        print("Description =", itemDescription)
        # File.write(f"{itemDescription},")
        data['description'].append(itemDescription)
        print("\n")

        #size 
        itemSize = size(itemTitle, itemDescription)
        data['size'].append(itemSize)

        #color
        itemColor = color(itemTitle, itemDescription)
        data['color'].append(itemColor)

        #material
        itemMaterial = material(itemTitle, itemDescription)
        data['material'].append(itemMaterial)

        #filter type
        itemFilter = filterType(itemTitle, itemDescription)
        data['filter type'].append(itemFilter)

        #gender
        itemGender = gender(itemTitle, itemDescription)
        data['gender'].append(itemGender)

        #disposability
        itemDisposability = disposability(itemTitle, itemDescription)
        data['disposability'].append(itemDisposability)

        #ear loop
        itemEarloop = earloop(itemTitle, itemDescription)
        data['earloop'].append(itemEarloop)

        # data[link] = [itemTitle, itemPrice, itemRating, itemRateCount, itemStock, itemDescription]

Streaming output truncated to the last 5000 lines.
Product Title = Bigox Face Mask Black 50Pcs
Product Price = 
Product Rating = 4.6 out of 5 stars
Number of Product Reviews = 10,377 ratings
Availability = In Stock.
Description =  Package: 50 Pcs black disposable face mask packaged in a box 3-ply design, with a melt-blown polypropylene filtration layer. How to use: Remember 1. Keep the black-side outside 2.Keep the nose clip on the upper side. You won't be wrong.


Product Title = Bigox Face Mask Purple 50Pcs
Product Price = 
Product Rating = 4.7 out of 5 stars
Number of Product Reviews = 1,458 ratings
Availability = In Stock.
Description =  Package: 50 Pcs purple disposable face mask packaged in a box 3-ply design, with a melt-blown polypropylene filtration layer. How to use: Remember 1. Keep the purple-side outside 2.Keep the nose clip on the upper side. You won't be wrong.


Product Title = Bigox Face Mask Disposable Earloop Blue 50Pcs
Product Price = 
Product Rating = 4.5 out of 5 

In [ ]:
df = pd.DataFrame(data)
print(df)

csvOut = df.to_csv('MasksWithFeatures_4_20_2021_225.csv', index = True)

                                                  link  ...     earloop
0    /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...     elastic
1    /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...          NA
2    /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...     elastic
3    /100pcs-Disposable-Breathable-Filter-Elastic/d...  ...     elastic
4    /Disposable-Effective-Filtration-Earloops-Outd...  ...     elastic
..                                                 ...  ...         ...
662  /Disposable-Breathable-Comfortable-Protection-...  ...     elastic
663  /Weddingstar-Washable-Reusable-Adjustable-Filt...  ...  adjustable
664  /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...     elastic
665  /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...          NA
666  /gp/slredirect/picassoRedirect.html/ref=pa_sp_...  ...          NA

[667 rows x 14 columns]


In [ ]:
!cat maskData.csv

cat: maskData.csv: No such file or directory
